In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [2]:
tf.__version__

'2.9.2'

In [3]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip',
                        dataset_url,
                        extract=True,
                        cache_dir='.')

dataframe = pd.read_csv(csv_file)

In [4]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [5]:
# Transform the problem into binary classification
# The `AdoptionSpeed` of `4` indicates
# a pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)
dataframe = dataframe.drop(columns=[
    'AdoptionSpeed',
    'Description'
])

In [6]:
train, val, test = np.split(
    dataframe.sample(frac=1),
    [int(0.8*len(dataframe)), int(0.9*len(dataframe))]
)

In [7]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

9229 training examples
1154 validation examples
1154 test examples


In [8]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {
        key: value.to_numpy()[:,tf.newaxis]
        for key, value in dataframe.items()
    }
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [9]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

Metal device set to: Apple M1 Pro


2022-07-25 23:34:10.664461: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-25 23:34:10.664554: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every features: ', list(train_features.keys()))
print('A batch of ages: ', train_features['Age'])
print('A batch of targets: ', label_batch)

Every features:  ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
A batch of ages:  tf.Tensor(
[[ 8]
 [12]
 [ 3]
 [ 2]
 [24]], shape=(5, 1), dtype=int64)
A batch of targets:  tf.Tensor([1 1 1 0 0], shape=(5,), dtype=int64)


### Apply the Keras preprocessing layers
The keras preprocessing layers allow you to build Keras-native input processing pipelines. They can be exported as part of a Keras SavedModel.

##### 1. Numerical columns

In [11]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature
    normalizer = layers.Normalization(axis=None)
    
    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [12]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

2022-07-25 23:34:10.750305: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-25 23:34:10.802523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:10.818565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.12097176],
       [-0.19530164],
       [ 7.39526   ],
       [-0.82784843],
       [-0.82784843]], dtype=float32)>

##### 2. Categorical columns

In [13]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a layer that turns strings into integer indices.
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    # Otherwise, create a layer that turns integer values into integer indices.
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
    
    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the set of possible values and assign them
    # a fixed integer index.
    index.adapt(feature_ds)
    
    # For debugging, vocabs will contain `[UNK]`
    # for StringLookup and -1 for IntegerLookup
#     print(index.get_vocabulary())
        
    # Encode the integer indices.
    encoder = layers.CategoryEncoding(
        num_tokens=index.vocabulary_size()
    )
    
    # Apply multi-hot encoding to the indices. The
    # lambda function captures the layer, so you
    # can use them, or include them in the Keras
    # Functional model later.
    return lambda feature: encoder(index(feature))

In [14]:
test_type_col = train_features['Type']
test_type_layer = get_category_encoding_layer(
    name='Type',
    dataset=train_ds,
    dtype='string'
)
test_type_layer(test_type_col)

2022-07-25 23:34:14.463504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)>

In [15]:
test_age_col = train_features['Age']
test_age_layer = get_category_encoding_layer(
    name='Age',
    dataset=train_ds,
    dtype='int64',
    max_tokens=5
)
test_age_layer(test_age_col)

2022-07-25 23:34:15.981298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)>

In [16]:
batch_size = 256
train_ds = df_to_dataset(
    train,
    batch_size=batch_size
)
val_ds = df_to_dataset(
    val,
    shuffle=False,
    batch_size=batch_size
)
test_ds = df_to_dataset(
    test,
    shuffle=False,
    batch_size=batch_size
)

In [17]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ['PhotoAmt', 'Fee']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

2022-07-25 23:34:17.868100: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:17.884033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.058807: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.075058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [18]:
age_col = tf.keras.Input(
    shape=(1,), name='Age', dtype='int64'
)

encoding_layer = get_category_encoding_layer(
    name='Age', dataset=train_ds,
    dtype='int64', max_tokens=5
)
encoded_age_col = encoding_layer(age_col)
all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

2022-07-25 23:34:18.239428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [19]:
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']

for header in categorical_cols:
    categorical_col = tf.keras.Input(
        shape=(1,), name=header, dtype='string'
    )
    encoding_layer = get_category_encoding_layer(
        name=header,
        dataset=train_ds,
        dtype='string',
        max_tokens=5
    )
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

2022-07-25 23:34:18.356019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.447116: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.538974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.631479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.722142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.812815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:18.904608: I tensorflow/core/grappler/optimizers/cust

In [20]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation='relu')(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [21]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=True
    ),
    metrics=['accuracy']
)

In [22]:
# Use `rankdir='LR'` to make the graph horizontal.
tf.keras.utils.plot_model(
    model, 
    show_shapes=True, 
    rankdir='LR'
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [23]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


/Users/bcp/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['target'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2022-07-25 23:34:19.720285: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


37/37 [==============================] - 2s 31ms/step - loss: 0.6683 - accuracy: 0.4616 - val_loss: 0.5765 - val_accuracy: 0.7010
Epoch 2/10


2022-07-25 23:34:20.881249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


37/37 [==============================] - 1s 15ms/step - loss: 0.5924 - accuracy: 0.6557 - val_loss: 0.5539 - val_accuracy: 0.7444
Epoch 3/10
37/37 [==============================] - 1s 14ms/step - loss: 0.5714 - accuracy: 0.6821 - val_loss: 0.5415 - val_accuracy: 0.7461
Epoch 4/10
37/37 [==============================] - 1s 14ms/step - loss: 0.5562 - accuracy: 0.7007 - val_loss: 0.5329 - val_accuracy: 0.7452
Epoch 5/10
37/37 [==============================] - 1s 14ms/step - loss: 0.5469 - accuracy: 0.7085 - val_loss: 0.5275 - val_accuracy: 0.7357
Epoch 6/10
37/37 [==============================] - 1s 14ms/step - loss: 0.5421 - accuracy: 0.7126 - val_loss: 0.5233 - val_accuracy: 0.7374
Epoch 7/10
37/37 [==============================] - 1s 15ms/step - loss: 0.5333 - accuracy: 0.7184 - val_loss: 0.5199 - val_accuracy: 0.7400
Epoch 8/10
37/37 [==============================] - 1s 14ms/step - loss: 0.5320 - accuracy: 0.7228 - val_loss: 0.5173 - val_accuracy: 0.7348
Epoch 9/10
37/37 [======

In [24]:
model.save('my_pet_classifier')
reloaded_model = tf.keras.models.load_model('my_pet_classifier')

INFO:tensorflow:Assets written to: my_pet_classifier/assets


INFO:tensorflow:Assets written to: my_pet_classifier/assets
2022-07-25 23:34:31.311100: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:31.316519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:31.321407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:31.326072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:31.331097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25 23:34:31.335890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-25

In [25]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)

1/1 [==============================] - 0s 333ms/step
This particular pet had a 77.1 percent probability of getting adopted.


2022-07-25 23:34:31.709876: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
